# Interactive COVID-19 data visualizations using Python & Bokeh

### Chris Coletta
### Friday, January 29, 2021

# Goals

* Reproduce many figures from the [New York Times U.S. tracking page](https://www.nytimes.com/interactive/2020/us/coronavirus-us-cases.html) and other sources
    * "Epidemic curve": Cumulative cases and deaths plotted by day
    * "[Chloropleth Map](https://en.wikipedia.org/wiki/Choropleth_map)": New cases and deaths plotted by state

* Answer some important questions:
    * Maryland: What is the county-by-county situation?
    * US: Is the increase in cases coming from an increase in testing, or from actual virus spread?

# Analysis Strategy

* Use one or more of the widely available APIs that provide COVID-19 data
* Download them programmatically so figures are up to date every time you run this notebook

## NIA IRP Data Viz workshop

* Close all anaconda windows
* Open up anaconda prompt
* Run the first few cells to make sure you have everything installed
    * ```!pip install pandas-bokeh```
    * ```!pip install geopandas```
        * Requires GDAL (On Mac: ```brew install gdal```)
* RAISE YOUR HAND IN ZOOM IF YOU HAVE DONE ALL OF THE ABOVE
* The notebook we will use today is in the Zoom chat box
* Two options for running:
    * Download and run locally.
    * OR run this notebook in Google CoLab

* Workshop participants are expected to run the code along with me
* Select ```Kernel->Restart & Clear Output``` to start
* Have my presentation side-by-side with your Jupyter Notebook open
* Unmute yourself if you have questions

* Can also use Jupyterlab, but you must run additional command and restart JupyterLab
    * ```jupyter labextension install @jupyter-widgets/jupyterlab-manager```
    * ```jupyter labextension install @bokeh/jupyter_bokeh```

# What we won't cover

* Effective reproduction number
    * [rt.live](https://rt.live) - state-by-state COVID-19 transmission rate $R_t$ modelling from the makers of Instagram

* More sophisticated epidemic modelling in general
    * Please refer to John Holmes' [brilliant talk](https://videocast.nih.gov/watch=37909) AI in the Age of COVID-19: Computational Tools for a Pandemic) from yesterday: "AI in the Age of COVID-19: Computational Tools for a Pandemic"

* Information retrieval from corpus of published and pre-published scientific manuscripts adressing coronavirus

# Available Data Sources

* [NYTimes COVID-19 data on GitHub](https://github.com/nytimes/covid-19-data)
    * Cases and deaths by US County
    * Excess deaths by time series data country

* [The COVID Tracking Project](https://covidtracking.com) at The Atlantic
    * Data broken down by race and other demographic factors/social determinants

* [CSSEGISandData COVID-19 data on GitHub](https://github.com/CSSEGISandData/COVID-19)
    * COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University
    * Includes normalized case, testing, and mortality rate per 100k

* [Humanitarian Data Exchange](https://data.humdata.org/dataset/total-covid-19-tests-performed-by-country)
    * UN website
    * COVID *testing* data by country

# Load the necessary libraries

* We use the [Pandas](https://pandas.pydata.org/) DataFrame as a container for the data, and also to load the data from internet URLs
* We use [pandas-bokeh](https://github.com/PatrikHlobil/Pandas-Bokeh) to create the interactive plots.
* We use [GeoPandas](https://geopandas.org/) to make the maps
* Optional: we use [requests](https://requests.readthedocs.io/en/master/) to download state and county shapes from the US census website

In [1]:
import pandas as pd

pd.set_option( 'display.max_rows', 100 )

pd.set_option( 'display.max_columns', 100 )

In [2]:
!pip install pandas_bokeh

In [3]:
import pandas_bokeh

#from bokeh.layouts import column, row
#from bokeh.models import CustomJS, Slider, Dropdown
#from bokeh.plotting import ColumnDataSource, figure, show
#from bokeh.io import output_notebook, output_file

In [4]:
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [5]:
!pip install geopandas

In [ ]:
import geopandas as gpd

In [6]:
import requests

In [7]:
import numpy as np

# Part 1: US Epidemic curves

* Rate of transmission is the slope of the epidemic curve
* Examples
    * [New York Times](https://www.nytimes.com/interactive/2020/us/coronavirus-us-cases.html) -  U.S. tracking page
    * [AP news](https://apnews.com/32cb14de009bfff91bc76a967323c621) - COVID-19 tracker with state pulldown
    * [Visual Capitalist](https://www.visualcapitalist.com/infection-trajectory-flattening-the-covid19-curve/) - Flattening the Curve by Country

# Download state-wide COVID time series

In [8]:
state_url = "https://covidtracking.com/api/v1/states/daily.csv"

In [9]:
state_data = pd.read_csv( state_url )

In [10]:
type( state_data )

pandas.core.frame.DataFrame

## Quick Review of PANDAS DataFrame capabilities

* Use period and tab key to get pop up list of methods and attributes
* Can always read the manual for each function by using the question mark

In [ ]:
state_data.

In [ ]:
state_data.shape

In [ ]:
state_data.info()

In [ ]:
state_data.head()

In [ ]:
state_data.tail(10)

### Optional: use ```to_csv()```method to save downloaded data to a csv

In [ ]:
state_data.to_csv( 'state_data.csv' )

### Select one column

Enclose the column label string in brackets ```[]```

In [ ]:
state_data[ 'state' ]

### Select one or more rows

Enclose the selection criteria in brackets ```[]```

In [ ]:
state_data[ state_data[ 'state' ] == 'MD' ]

### ```groupby()``` operation example

Sum all daily deaths since the beginning of the pandemic on a state-by-state basis, and sort in descending order

### Using ```groupby()```, ```sum()```, and ```sort_values()``` method in a single line of code

1. Group the rows by the value in the ```'state'``` column.
2. For each batch of rows:
    1. Take the values in the ```'deathIncrease'``` column
    2. Sum them, returning a new DataFrame that has one number per state.
2. Sort the values in descending order

In [ ]:
state_data.groupby( 'state' )[ 'deathIncrease' ].sum().sort_values( ascending=False )

In [ ]:
state_data.groupby( 'state' )[ 'deathIncrease' ].sum().sort_values( ascending=False )

### CAVIAT: Understanding the data types of the columns is key

* string, integer, float, datatime, etc

In [ ]:
state_data.groupby( 'date' )[ "death" ].sum().plot()

### PANDAS is interpreting the date column as an integer

In [ ]:
state_data[ 'date' ]

### Use ```pd.to_datetime()``` function to reinterpret dates

Remember ```pd``` is a shortcut for ```pandas``` because we ```import pandas as pd```

In [ ]:
state_data[ 'date' ] = pd.to_datetime( state_data[ 'date' ], format="%Y%m%d" )

In [ ]:
state_data[ 'date' ]

# Quick (non-interactive) plot #1
## Cumulative number of COVID-19 *deaths* in the US

In [ ]:
state_data.groupby( 'date' )[ "death" ].sum().plot()

### In this case, the x and y axes were inferred from the structure of the DataFrame

In [ ]:
state_data.groupby( 'date' )[ "death" ].sum()

# Quick (non-interactive) plot #2
## Daily US COVID-19 *cases* increase

In [ ]:
state_data.groupby( 'date' )[ "positiveIncrease" ].sum().plot()

# Interactive plot #1
## Daily US COVID-19 *cases* increase with 7-day rolling average
### Strategy:

1. Construct a new DataFrame with two columns
    1. Raw daily increase
    2. Rolling daily increase
2. Use the ```plot_bokeh()``` function to generate the interactive plot

### Store daily increase in a variable

In [ ]:
us_new_cases_daily = state_data.groupby( 'date' )[ "positiveIncrease" ].sum()

In [ ]:
us_new_cases_daily

In [ ]:
us_new_cases_daily.name = "Daily increase"

### Use the ```rolling()```method to create smooth data series

In [ ]:
# can also pass "center=True" argument to rolling()
us_new_cases_7day_mean = us_new_cases_daily.rolling(7).mean()

In [ ]:
us_new_cases_7day_mean

### Round the smoothed data to the nearest person

In [ ]:
us_new_cases_7day_mean = us_new_cases_daily.rolling(7).mean().fillna(0).round().astype( int )

In [ ]:
us_new_cases_7day_mean.name = '7-day average'

In [ ]:
us_new_cases_7day_mean

### Use ```pandas.concat()``` to bind columns together

In [ ]:
new_cases_df = pd.concat( ( us_new_cases_daily, us_new_cases_7day_mean ), axis='columns' )

# Comparison of quick plot and interactive plot

In [ ]:
new_cases_df.plot()

In [ ]:
new_cases_df.plot_bokeh( legend="top_left" )

# Saving plot to HTML file

I have been using a somewhat hacky workaround to save plots. I use the ```return_html=True``` argument to grab the raw html, and writing that directly to a file.

In [ ]:
html = new_cases_df.plot_bokeh( legend="top_left", return_html=True )

In [ ]:
with open("/Users/colettace/Desktop/CHRIS_BOKEH_PLOT.html" , "w") as f:
    f.write( '<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n' )
    f.write( '<html lang="en">\n' )
    f.write( '  <head>\n' )
    f.write( '    <meta http-equiv="content-type" content="text/html; charset=utf-8">\n' )
    f.write( '    <title>CHRIS\'S BOKEH PLOT</title>\n' )
    f.write( '  </head>\n' )
    f.write( '  <body>\n' )
    f.write( html )
    f.write( '  </body>\n' )
    f.write( '</html>\n' )

# Part 2: Maryland county-level chloropleth map

In [ ]:
us_county_covid_death_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
us_county_covid_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"

In [ ]:
us_county_covid_data = pd.read_csv( us_county_covid_url )

In [ ]:
type( us_county_covid_data )

In [ ]:
us_county_covid_data.shape

In [ ]:
us_county_covid_data.head()

In [ ]:
us_county_covid_data.sample(10)

In [ ]:
wanted_rows = us_county_covid_data['Province_State'] == 'Maryland'

In [ ]:
wanted_rows

In [ ]:
maryland_county_covid_data = us_county_covid_data[ wanted_rows ].copy()

In [ ]:
maryland_county_covid_data.shape

In [ ]:
maryland_county_covid_data

## Download US geographic boundaries from www.census.gov

* [US geographic boundaries download page](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html)
    * Manually download and unzip [1:500,000 scale shapefiles](https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_us_county_500k.zip)
    * Use Python to download the file and unzip it programmatically

In [ ]:
url = 'https://www2.census.gov/geo/tiger/GENZ2019/shp/cb_2019_us_county_500k.zip'
r = requests.get(url, allow_redirects=True)
open('cb_2019_us_county_500k.zip', 'wb').write(r.content)

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('cb_2019_us_county_500k.zip') as myzip:
    myzip.extractall()

In [ ]:
us_county_shapes = gpd.read_file( 'cb_2019_us_county_500k.shp' )

In [ ]:
type( us_county_shapes )

In [ ]:
us_county_shapes.shape

In [ ]:
us_county_shapes.info()

In [ ]:
us_county_shapes.head()

# Join the COVID table and the geography table together

Create a table join key 'FIPS' [Federal Information Processing Standard state code](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code)

In [ ]:
us_county_shapes['FIPS'] = us_county_shapes['STATEFP'] + us_county_shapes['COUNTYFP'] 

In [ ]:
us_county_shapes.head()

## Make sure the FIPS column has the same data type

In [ ]:
us_county_shapes['FIPS'].dtype

In [ ]:
maryland_county_covid_data['FIPS'].dtype

In [ ]:
us_county_shapes['FIPS'] = us_county_shapes['FIPS'].astype( int )

In [ ]:
maryland_county_covid_data['FIPS'] = maryland_county_covid_data['FIPS'].astype( int )

## Perform table merge

* INNER join: resultant output contains only the rows for which the join keys appear in both tables

In [ ]:
merged_data = us_county_shapes.merge( maryland_county_covid_data, on='FIPS', how='inner' ) 

In [ ]:
merged_data

# Chloropleth Map

* Use ```plot_bokeh()```
* IMPORTANT:
    * Chloropleth maps require the data frame object to be a ```GeoDataFrame```
    * A column with label ```'geometry'``` must be in the ```GeoDataFrame```

In [ ]:
type( merged_data )

In [ ]:
'geometry' in merged_data.columns

In [ ]:
merged_data.plot_bokeh()

In [ ]:
date_columns = [ _ for _ in merged_data.columns if _.endswith( '/20' ) ]

In [ ]:
len( date_columns )

In [ ]:
date_columns[-10:]

In [ ]:
merged_data.plot_bokeh(
    #figsize=(1600, 900),
    #figsize=(600, 400),
    slider=date_columns,
    slider_name="Date",
    hovertool_columns= ['NAME'] + date_columns[-5:] )

In [ ]:
gpd.__version__